In [2]:
import healpy as hp
import itertools
import jax
import jax.numpy as jnp
from jax import vmap, grad
from response import pairwise_monopole, pairwise_dipole, vvec,dir,pulsar_star_pair_monopole, pulsar_star_pair_dipole
from jax import config
# config.update("jax_enable_x64", True)
import numpy as np
# plot settings
import matplotlib
import matplotlib.pyplot as plt
np.set_printoptions(precision=4,suppress=True)
jnp.set_printoptions(precision=4,suppress=True)

jax.config.update("jax_enable_x64", True)

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [12]:
#vvec = np.array([0.6, 0.4, 0.69282]).T #For testing the code
#vvec = dir(theta0,phi0)
pi = np.pi

nside = 128
npix = hp.nside2npix(nside)
pix = np.arange(npix)
pvec_array = np.array(hp.pix2vec(nside,pix)).T

#nvec = np.array([1, 0, 0]).T
#xvec = np.array([1/2, 0, (np.sqrt(3)/2)]).T

vvec1 = vvec
nvec1 = vvec
#nvec1 = np.array([0.96388439, -0.26566278, 0.01871302]).T
xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T
#xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T

dipole = pulsar_star_pair_dipole(nvec1,xvec1,pvec_array,nside)

print("Numerical dipole = ", dipole)


#### The next vectors are for testing the analytical expressions

vvec1 = vvec 
#vvec1 = np.array([0.6, 0.4, 0.69282]).T
#nvec1 = np.array([0, 0, 1]).T
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T

nvec1 = vvec # This one gives a different result than the numerical solution
#nvec1 = np.array([0.96388439, -0.26566278, 0.01871302]).T
xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T


def dipole_analytical(nvec,xvec,vvec=vvec1):
    ndotx = jnp.sum(nvec*xvec)
    y = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)

    A1 = jnp.cross(nvec, xvec)
    A2 = jnp.cross(nvec, vvec)
    A1dotv = jnp.sum(A1*vvec)
    A1dotA2 = jnp.sum(A1*A2)

    def get_a1():
        num1 = jnp.pi*( (A1dotv**2)*(1 - 12*y) + A1dotA2*((A1dotA2) + 12*(A1dotA2)*(y**2) + 4*(ndotv)*y*(1 + y*(5 - 6*y)) )) 
        den1 = 6*A1dotv*(1 - y)*y
        num2 = 2*np.pi*( (A1dotA2**2) - (A1dotv**2) + 2*A1dotA2 *(ndotv)* (1 - y))*y*jnp.log(y)
        den2 = A1dotv* ((1-y)**2)
        return (num1/den1) + (num2/den2) 

    def get_a2():
        num3 = -2*jnp.pi*( A1dotA2 + 12*A1dotA2*(y**2)  + 4*ndotv*y*(1 + y*(5 - 6*y)) ) 
        den3 = 3*A1dotv
        num4 = -8*jnp.pi*( A1dotA2 + 2*(ndotv)*(1 - y) ) *((y**2) *jnp.log(y))
        den4 = A1dotv*(1 - y)
        return (num3/den3) + (num4/den4)       

    a1 = get_a1()
    a2 = get_a2()
    #print('a1, a2:', a1,a2)
    #print('A1, A2:', A1,A2)
    K_i = (a1*A1) + (a2*A2)
    return K_i

def dipole_analytical_Vparallel(nvec,xvec,vvec=vvec1):
    ndotx = jnp.sum(nvec*xvec)
    z = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)
    
    term1 = 2*np.pi*(nvec*(1 - 2*z) - xvec)/ (3* (1 - z))
    term2 = ((z - 1)*(6*z + 1) - 6*z* np.log(z))
    return term1*term2


dipole_gt = dipole_analytical(nvec1,xvec1)

dipole_GTvParallel = dipole_analytical_Vparallel(nvec1,xvec1)

print("Analytical dipole = ",dipole_gt)
print("Analytical dipole = ",dipole_GTvParallel)
print("K1K1_Analytical dipole = ",jnp.dot(dipole_gt, dipole_gt))
print("dipole_Squared = ", np.dot(dipole_gt,dipole_gt))
print("dipole_Squared = ", jnp.sum(dipole_gt*dipole_gt))
print("dipole_Squared = ", (dipole_gt.T)@(dipole_gt))



Numerical dipole =  [-0.0304 -0.4989 -0.4496]
Analytical dipole =  [-0.0442 -0.5439  0.0142]
Analytical dipole =  [-0.0304 -0.4989 -0.4496]
K1K1_Analytical dipole =  0.2979741789110825
dipole_Squared =  0.2979741789110825
dipole_Squared =  0.2979741789110825
dipole_Squared =  0.2979741789110825


In [3]:
m = 8.890394245545021*np.array([0, 0.5, 0,])
n = -6.916943974026755*np.array([-0.4,  0.6,  0. ])

print(m+n)

[2.7668 0.295  0.    ]


# Stars-Stars correlation

In [18]:
#vvec1 = np.array([0.6, 0.4, 0.69282]).T
#nvec1 = np.array([1, 0, 0]).T
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T

vvec1 = vvec
#vvec1 = np.array([0.6, 0.4, 0.69282]).T
#nvec1 = np.array([0, 0, 1]).T
#xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T

nvec1 = -0.99999*vvec
#nvec1 = np.array([0.96388439, -0.26566278, 0.01871302]).T
#xvec1 = np.array([9.61440915e-04, -2.51119328e-01, -9.67955659e-01]).T
xvec1 = np.array([1/2, 0, (np.sqrt(3)/2)]).T


#theta0 = np.deg2rad(48) 
#phi0 = np.deg2rad(264)
#vvec1 = dir(theta0,phi0)

#theta1 = np.pi/3
#phi1 =  np.pi/4
#theta2 = np.pi/8
#phi2 = np.pi/6 # pi/20

#nvec1 = dir(theta1,phi1)
#xvec1 = dir(theta2,phi2)

#nside = 128
#npix = hp.nside2npix(nside)
#pix = np.arange(npix)
#pvec_array = np.array(hp.pix2vec(nside,pix)).T

Hdipole = pairwise_dipole(nvec1,xvec1,pvec_array,nside)

print("(SS) Numerical dipole = ",Hdipole)


def Hdipole_analytical(nvec,xvec,vvec=vvec1):
    ndotx = jnp.sum(nvec*xvec)
    y = (1-ndotx)/2
    ndotv = jnp.sum(nvec*vvec)
    xdotv = jnp.sum(xvec*vvec)
    
    A = jnp.cross(nvec,xvec)
    B = jnp.cross(nvec, A)
    C = -jnp.cross(xvec, A)
    Adotv = jnp.sum(A*vvec)
    AdotA = jnp.sum(A*A)
    BdotB = jnp.sum(B*B)
    CdotC = jnp.sum(C*C)

    def get_a1():
        num = 8 *jnp.pi*(Adotv)*y* ( (3*(y**2)*jnp.log(y) / (y - 1)) - 4*y + 1)
        den = 3*(AdotA)*(CdotC)
        return num/den

    def get_a2():
        num = 8*jnp.pi*(Adotv)*y*( (3*(y**2)*jnp.log(y) / (y - 1)) - 4*y + 1)
        den = 3*(AdotA)*(BdotB)
        return num/den
        
    def get_a3():
        num = -8*jnp.pi*(y**2)*( (y - 1)*(2*y + 1) - 3*y*jnp.log(y)) * (2*ndotv*(y - 1) - jnp.sqrt( 4*( ndotv**2 - 1)*(y - 1)*y - Adotv**2))
        den = 3*(y - 1)*(AdotA**2)
        return num/den 

    def get_a4():
        num = (8*np.pi*(y**2)*( (y - 1)*(2*y + 1) - 3*y*jnp.log(y)) * 
          ( np.sqrt( ((Adotv**2)/(y - 1)) - 4*(ndotv**2 - 1)*y ) + 2*ndotv*np.sqrt(1 - y)))
        den = 3*np.sqrt(1 - y)*BdotB*CdotC
        return (num/den) 

    a1 = get_a1()
    a2 = get_a2()
    a3 = get_a3()
    a4 = get_a4()
    #print('a1, a2:', a1,a2)
    print('AdotA:', AdotA)
    H_ij = a1*np.outer(A, C) + a2*np.outer(B, A) + a3*np.outer(A, A) + a4*np.outer(B, C)
    return H_ij

Hdipole_gt = Hdipole_analytical(nvec1,xvec1)

print("(SS) Analytical dipole = ",Hdipole_gt)


(SS) Numerical dipole =  [[ 0.0529 -0.25   -0.0305]
 [-0.1642 -0.0339  0.0948]
 [-0.142  -0.0539  0.082 ]]
AdotA: 0.6295804233782628
(SS) Analytical dipole =  [[ 0.0524 -0.2477 -0.0303]
 [-0.1627 -0.0336  0.0939]
 [-0.1407 -0.0534  0.0813]]
